In [1]:
# matrix with all the gene expression values for a particular condition 

genes, gene lengths, 
observed- gene expressionn values
for neg bino reg model - need parameter for mean expression level and dispersion parameter 
mean expression level can be stratified by catogery

In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
# These MUST come before importing pymc or pytensor
os.environ["PYTENSOR_FLAGS"] = "optimizer_excluding=constant_folding,mode=FAST_RUN,linker=py"
os.environ["PYTENSOR_WARN__C_COMPILE"] = "False"

import pytensor
pytensor.config.cxx = ""

# Now import pymc
import pymc as pm
import arviz as az
print("Linker:", pytensor.config.linker)

import numpy as np
import pandas as pd
import scipy.sparse as sp

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Linker: py


In [2]:
import scanpy as sc
import matplotlib.pyplot  as plt

In [3]:
import pymc as pm
print("PyMC version:", pm.__version__)


PyMC version: 5.12.0


In [4]:
adata = sc.read("merged_dataset.h5ad")

In [5]:
# Print a summary of the dataset
print(adata)

AnnData object with n_obs × n_vars = 186224 × 39241
    obs: 'AvgSpotLen', 'Bases', 'Bytes', 'original_index'


In [6]:
# Fill NaNs in obs['Bases']
adata.obs["Bases"] = adata.obs["Bases"].fillna(0)

In [7]:
n_cells = adata.shape[0]
n_genes = adata.shape[1]

In [8]:
gene_lengths = np.random.uniform(1000, 3000, size=n_genes)
seq_depth = np.median(adata.obs['Bases'].values) if 'Bases' in adata.obs.columns else 1e4
ref_seq_depth = 1e5
ref_gene_length = 2000.0
batch_size = 4096

In [9]:
# Prepare output shape: (n_genes, n_cells)
observed_counts = np.zeros((n_genes, n_cells), dtype=np.int32)

In [10]:
for start in range(0, n_cells, batch_size):
    end = min(start + batch_size, n_cells)
    # Get cell-wise batch
    X_batch = adata.X[start:end]

    # Convert batch to dense safely
    X_batch = X_batch.toarray() if sp.issparse(X_batch) else np.asarray(X_batch)

    # Store in observed_counts
    observed_counts[:, start:end] = X_batch.T.astype(np.int32)

In [12]:
n_genes, n_conditions = observed_counts.shape

In [13]:
import scipy.sparse as sp

def generate_minibatch_indices_from_adata(adata, batch_size=8192):
    n_genes, n_cells = adata.shape
    total_size = n_genes * n_cells

    for start in range(0, total_size, batch_size):
        end = min(start + batch_size, total_size)

        gene_idx = np.arange(start, end) // n_cells
        cond_idx = np.arange(start, end) % n_cells

        # Load only necessary columns per minibatch
        cols_needed = np.unique(cond_idx)
        adata_chunk = adata[:, cols_needed]

        # Convert sparse to dense
        X_chunk = adata_chunk.X.toarray() if sp.issparse(adata_chunk.X) else adata_chunk.X

        # Pull count values based on local cond_idx
        cond_local = np.searchsorted(cols_needed, cond_idx)
        counts = X_chunk[gene_idx, cond_local]

        yield {
            "counts": counts.astype(np.int32),
            "gene_idx": gene_idx.astype(np.int32),
            "cond_idx": cond_idx.astype(np.int32),
        }


In [14]:
init_batch = next(generate_minibatch_indices_from_adata(adata, batch_size=4096))

In [15]:
counts = init_batch['counts']
gene_idx = init_batch['gene_idx']
cond_idx = init_batch['cond_idx']

In [16]:
flat_gene_idx = gene_idx.flatten()
flat_cond_idx = cond_idx.flatten()

In [ ]:
# --------------------------
# PyMC Model
# --------------------------
with pm.Model() as model_vi:
    counts_shared = pm.MutableData("counts_shared", init_batch["counts"])
    gene_idx_shared = pm.MutableData("gene_idx_shared", init_batch["gene_idx"])
    cond_idx_shared = pm.MutableData("cond_idx_shared", init_batch["cond_idx"])
    gene_lengths_shared = pm.Data("gene_lengths_shared", gene_lengths)

    mu_gene = pm.Normal("mu_gene", mu=2.0, sigma=1.0)
    sigma_gene = pm.HalfNormal("sigma_gene", sigma=1.0)
    z_gene = pm.Normal("z_gene", mu=0, sigma=1, shape=n_genes)
    gene_log_expression = pm.Deterministic("gene_log_expression", mu_gene + sigma_gene * z_gene)

    mu_cond = pm.Normal("mu_cond", mu=0.0, sigma=1.0)
    sigma_cond = pm.HalfNormal("sigma_cond", sigma=1.0)
    z_cond = pm.Normal("z_cond", mu=0, sigma=1, shape=n_cells)
    condition_intercepts_log = pm.Deterministic("condition_intercepts_log", mu_cond + sigma_cond * z_cond)

    alpha = pm.HalfCauchy("alpha", beta=2)

    log_mu = gene_log_expression[gene_idx_shared] + condition_intercepts_log[cond_idx_shared]
    log_mu += np.log(seq_depth / ref_seq_depth)
    log_mu += pm.math.log(gene_lengths_shared[gene_idx_shared] / ref_gene_length)
    mu = pm.Deterministic("mu", pm.math.exp(log_mu))

    pm.NegativeBinomial("obs", mu=mu, alpha=alpha, observed=counts_shared)

    approx = pm.fit(n=1, method="advi", progressbar=False)

# --------------------------
# Manual VI Loop (stable)
# --------------------------
with model_vi:
    elbos = []
    gen = generate_minibatch_indices_from_adata(adata, batch_size=batch_size)

    for i in range(10000):  # tune based on time
        try:
            batch = next(gen)
        except StopIteration:
            gen = generate_minibatch_indices_from_adata(adata, batch_size=batch_size)
            batch = next(gen)

        pm.set_data({
            "counts_shared": batch["counts"],
            "gene_idx_shared": batch["gene_idx"],
            "cond_idx_shared": batch["cond_idx"],
        })

        approx = pm.fit(n=1, method="advi", progressbar=True)
        elbos.append(approx.hist[-1])

        if i % 100 == 0:
            print(f"Step {i}, ELBO: {elbos[-1]:.2f}")

    trace_vi = approx.sample(1000)

/scratch/hr2547/myenv/lib64/python3.9/site-packages/pymc/data.py:434: UserWarning: The `mutable` kwarg was not specified. Before v4.1.0 it defaulted to `pm.Data(mutable=True)`, which is equivalent to using `pm.MutableData()`. In v4.1.0 the default changed to `pm.Data(mutable=False)`, equivalent to `pm.ConstantData`. Use `pm.ConstantData`/`pm.MutableData` or pass `pm.Data(..., mutable=False/True)` to avoid this warning.
  warnings.warn(
/scratch/hr2547/myenv/lib64/python3.9/site-packages/pytensor/tensor/rewriting/elemwise.py:1030: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(
Finished [100%]: Loss = 1.4478e+05
/scratch/hr2547/myenv/lib64/python3.9/site-packages/pytensor/tensor/rewriting/elemwise.py:1030: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(


Finished [100%]: Loss = 1.1512e+05


Step 0, ELBO: 115123.54


/scratch/hr2547/myenv/lib64/python3.9/site-packages/pytensor/tensor/rewriting/elemwise.py:1030: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(


Finished [100%]: Loss = 2.2347e+05
/scratch/hr2547/myenv/lib64/python3.9/site-packages/pytensor/tensor/rewriting/elemwise.py:1030: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(


...

Finished [100%]: Loss = 92,075
/scratch/hr2547/myenv/lib64/python3.9/site-packages/pytensor/tensor/rewriting/elemwise.py:1030: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(


Finished [100%]: Loss = 1.1733e+05
/scratch/hr2547/myenv/lib64/python3.9/site-packages/pytensor/tensor/rewriting/elemwise.py:1030: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(


Finished [100%]: Loss = 1.4131e+05
/scratch/hr2547/myenv/lib64/python3.9/site-packages/pytensor/tensor/rewriting/elemwise.py:1030: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(


Finished [100%]: Loss = 1.0391e+05
/scratch/hr2547/myenv/lib64/python3.9/site-packages/pytensor/tensor/rewriting/elemwise.py:1030: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(


Finished [100%]: Loss = 1.4751e+05


In [ ]:
az.to_netcdf(trace_vi, "trace_vi.nc")

In [ ]:
az.summary(trace_vi)

In [ ]:
# --------------------------
# ELBO Plot for VI
# --------------------------
plt.plot(approx.hist)
plt.title("ELBO Loss Curve (VI)")
plt.xlabel("Iteration")
plt.ylabel("ELBO")
plt.tight_layout()
plt.show()

In [ ]:
# --------------------------
# Posterior Analysis
# --------------------------
az.plot_trace(trace_vi, var_names=["mu_gene", "sigma_gene", "mu_cond", "sigma_cond", "alpha"])
plt.tight_layout()
plt.show()